In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import os
import json 

import xgboost
from sklearn.metrics import r2_score

In [0]:
from google.colab import drive 
drive.mount('/mntDrive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
PREP_DATA_PATH = "/mntDrive/My Drive/НИР/data/prepared_facebook_data.csv"
data=pd.read_csv(PREP_DATA_PATH)

RES_SAVE_DIR = "/mntDrive/My Drive/НИР/data"

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# get some data
X, y = data.iloc[:,:-1].values, data.iloc[:,-1].values

In [0]:
dtrain = xgboost.DMatrix(X, label=y)

In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 4.4MB/s 


In [0]:
from skopt import BayesSearchCV
from skopt.space import Real, Space, Integer

bayes_cv_opt = BayesSearchCV(
    estimator = xgboost.XGBRegressor(
      tree_method = "gpu_hist", 
      gpu_id = 0, 
      verbosity = 0
    ),
    search_spaces = {
        'min_child_weight': Integer(1, 10),
        'max_depth': Integer(3, 10),
        'subsample': Real(0.5, 0.9, 'uniform'),
        'colsample_bytree': Real(0.5, 0.9, 'uniform'),
        'reg_lambda': Real(0.0, 1.0, 'uniform'),
        'reg_alpha': Real(0.0, 1.0, 'uniform'),        
    },    
    scoring = 'r2',
    cv = 5,
    n_iter = 729,
    iid=False, 
    optimizer_kwargs={
        "acq_func": "EI",
        "random_state":42,
        # n_initial_points
        }
)

# callback handler

def on_step(optim_result):
    try: 
      iteration_num += 1
      print(iteration_num)
    except:
      iteration_num = 1
      print(iteration_num)
    score = bayes_cv_opt.best_score_
    print("Best score: %s" % score)
    print("Best iteration index ", bayes_cv_opt.best_index_)
    #if score >= 0.98:
    #    print('Interrupting!')
    #    return True

bayes_cv_opt.fit(X, y, callback=on_step)

print("val. score: %s" % bayes_cv_opt.best_score_)


Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score: 0.6618210459312518
Best iteration index  0
Best score

In [0]:
res=pd.DataFrame(bayes_cv_opt.cv_results_)
res['run_number']=1
res['experiment_name']="bayes_opt"
res.to_csv("/mntDrive/My Drive/НИР/bayes_opt_1.csv", index=False)